In [ ]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.Control;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Benchmark" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "dbname" : dbname;
string table_name = String.Concat(buildname, "_", dbname);

In [ ]:
//var SubTab = TableExtensions.LoadFromFile(@"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\XdgPoisson\data_weak\Xdg_weakscaling_s_2020-9-9.json");

In [ ]:
string CurrentDocDir = Directory.GetCurrentDirectory();
string path = String.Concat(CurrentDocDir,@"/",table_name,".json");
var SubTab = TableExtensions.LoadFromFile(path);

Error: System.IO.FileNotFoundException: Could not find file 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\XdgPoisson\run_17-1-2022_Benchmark.json'.
File name: 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\XdgPoisson\run_17-1-2022_Benchmark.json'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path, Encoding encoding, Boolean detectEncodingFromByteOrderMarks)
   at System.IO.File.InternalReadAllText(String path, Encoding encoding)
   at System.IO.File.ReadAllText(String path)
   at BoSSS.Application.BoSSSpad.TableExtensions.LoadFromFile(String fil

In [ ]:
SubTab.GetColumnNames().OrderBy(s=>s);

In [ ]:
SubTab.Rows.Count

13

In [ ]:
// downward compatibility ...
string LSolverCode = SubTab.Columns.Contains("LinearSolver.SolverCode")? "LinearSolver.SolverCode" : "LinearSolverCode";

In [ ]:
var tab                        = SubTab.ExtractColumns(LSolverCode, "DOFs","DGdegree:u","maxSolRunT","ProjectName","SessionName","NoIter");
System.Data.DataView dv        = tab.DefaultView;
dv.Sort                        = LSolverCode+",DOFs";
System.Data.DataTable sortedDT = dv.ToTable();
sortedDT.Print();

    LinearSolver.SolverCode DOFs    DGdegree:u maxSolRunT         ProjectName           SessionName                          NoIter 
0:  2                       43840   2          40.9736952         Xdg_weakscaling       J4096_k2_classic_pardiso             1      
1:  2                       84320   2          65.7982942         Xdg_weakscaling       J8000_k2_classic_pardiso             1      
2:  2                       183200  2          170.00860459999998 Xdg_weakscaling       J17576_k2_classic_pardiso            1      
3:  2                       339040  2          527.6745040999999  Xdg_weakscaling       J32768_k2_classic_pardiso            1      
4:  2                       660160  2          2208.329985        Xdg_weakscaling       J64000_k2_classic_pardiso            1      
5:  41                      43840   2          183.4011587        Xdg_weakscaling       J4096_k2_exp_Kcycle_schwarz          14     
6:  41                      84320   2          515.5230200999999  Xdg

In [ ]:
Func<int, PlotRowSelector>  Solver_DG_Proj = delegate(int Degree){
  return delegate (int iTabRow, 
            IDictionary<string, object> Row, 
            out string Nmn, 
            out PlotFormat Fmt){

      Nmn = "";
      Fmt = new PlotFormat();
                
      
    
      int SolverCode = Convert.ToInt32(Row[LSolverCode]);
      int DGdegree   = Convert.ToInt32(Row["DGdegree:u"]);
      string PrjName = Convert.ToString(Row["ProjectName"]);

      switch(SolverCode){
          case 41:
              Nmn+="kcycle";
              //Fmt.LineColor = LineColors.Blue;
              Fmt.PointType = PointTypes.Asterisk;
              break;
          case 47:
              Nmn+="gmres";
              //Fmt.LineColor = LineColors.Yellow;
              Fmt.PointType = PointTypes.UpperTriangle;
              break;
          case 2:
              Nmn+="pardiso";
              //Fmt.LineColor = LineColors.Red;
              Fmt.PointType = PointTypes.OpenCircle;
              Fmt.DashType  = DashTypes.Dotted;
              break;
          default:
              Nmn = null;
              Fmt = null;
          return;
      }
      
      
      if(DGdegree!=Degree){
          Nmn = null;
          Fmt = null;
          return;
      }
      switch(DGdegree){
          case 2:
            Nmn+=" DG2";
            Fmt.PointType = PointTypes.OpenDiamond;
            break;
          case 3:
            Nmn+=" DG3";
            Fmt.PointType = PointTypes.OpenDiamond;
            break;
          case 5:
            Nmn+=" DG5";
            Fmt.PointType = PointTypes.OpenDiamond;
            break;

          default:
            Nmn = null;
            Fmt = null;
            return;
      }
  //           switch(PrjName){
  //               case "SIP_weakscale":
  //                Nmn +=" openmp";
  //                Fmt.LineColor = LineColors.Blue;
  //                break;
  //               case "SIP_weakscale_SEQ":
  //                Nmn += " seq";
  //                Fmt.LineColor = LineColors.Yellow;
  //                break;
  //               default:
  //                Nmn = null;
  //                Fmt = null;
  //                return;
  //           }
        
      Fmt.PointSize = 0.5;
      Fmt.LineWidth = 3;
      Fmt.Style     = Styles.LinesPoints;
      Fmt.DashType  = DashTypes.Solid;
  };
};

In [ ]:
// ==========
// parameterz
// ==========
int DG   = 2;
int xMin = 4;
int xMax = 512;
// ======================
// print runtime vs cores
// ======================
Plot2Ddata[,] multiplots = new Plot2Ddata[2,1];
multiplots[0,0] = SubTab.ToPlot("NoOfCores","maxSolRunT",Solver_DG_Proj(DG));
multiplots[0,0].LogX = true;
multiplots[0,0].LogY = true;
multiplots[0,0].tmargin = 0;
multiplots[0,0].bmargin = 2;
multiplots[0,0].XrangeMin=xMin;
multiplots[0,0].XrangeMax=xMax;
multiplots[0,0].Ylabel = "~wallclock time";
multiplots[0,0].LegendAlignment=new string[]{"i","b","r"};
// ==============
// print 2h limit
// ==============
double[] xaxis = new[]{(double)xMin,(double)xMax};
double[] T = xaxis.Select(x => 7200.0).ToArray();
var linT              = new Plot2Ddata.XYvalues("2h limit", xaxis, T);
linT.Format.FromString("- black");
linT.Format.LineWidth = 2;
linT.Format.DashType  = DashTypes.DotDashed;
ArrayTools.AddToArray(linT, ref multiplots[0,0].dataGroups);
// =====================
// print noiter vs cores
// =====================
multiplots[1,0] = SubTab.ToPlot("NoOfCores","NoIter",Solver_DG_Proj(DG));
multiplots[1,0].LogX = true;
multiplots[1,0].LogY = true;
multiplots[1,0].tmargin = 0;
multiplots[1,0].bmargin = 3;
multiplots[1,0].XrangeMin=xMin;
multiplots[1,0].XrangeMax=xMax;
multiplots[1,0].Xlabel = "no of cores";
multiplots[1,0].Ylabel = "iterations";
multiplots[1,0].LegendAlignment=new string[]{"i","b","r"};
multiplots.PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside bottom right Left reverse 
set key font ",16"inside bottom right Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 w a llclock time 
 
 
 
 
 kcycle DG2 
 
 
 
 
 kcycle DG2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 pardiso DG2 
 
 
 pardiso DG2 
 
 
 
 
 
 
 
 
 
 
 
 2h limit 
 
 
 
 
 2h limit 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iterations 
 
 
 
 
 no of cores 
 
 
 
 
 kcycle DG2 
 
 
 
 
 kcycle DG2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 pardiso DG2 
 
 
 pardiso DG2

In [ ]:
/*
string plotsdirpath = CurrentDocDir+@"\"+"plots";
DirectoryInfo dir   = Directory.CreateDirectory(plotsdirpath);
var plot2print      = multiplots;
var CL              = plot2print.ToGnuplot().PlotCairolatex(xSize:14,ySize:12);
CL.WriteMinimalCompileableExample(
   Path.Combine(plotsdirpath, "Scaling_"+DG+"_"+".tex"),
   "Scaling_"+DG+".tex");
   */

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside bottom right Left reverse 
set key font ",16"inside bottom right Left reverse 
